In [186]:
from math import *
import numpy as np

def machine_epsilon(func=float):
    '''
    Вспомогательная функция для вычисления машинного нуля
    '''
    machine_epsilon = func(1)
    while func(1)+func(machine_epsilon) != func(1):
        machine_epsilon_last = machine_epsilon
        machine_epsilon = func(machine_epsilon) / func(2)
    return machine_epsilon_last

def eval_Lm(x, n):
    '''
    Вычисление полинома Лежандра n-ой степени
    '''
    res = None
    if n > 1:
        Lm0 = 1 #значение полинома 0-й степени
        Lm1 = x #значение полинома 1-й степени
        lim_n = 2
        while True:
            res = (
                ((2*lim_n-1)*x*Lm1 - (lim_n-1)*Lm0)
                /lim_n
            )
            Lm0 = Lm1
            Lm1 = res
            if lim_n == n:
                break
            lim_n += 1
        return res
    else:
        return 1 if n == 0 else x

def __eval_Lm(x, n):
    if n < 2:
        return 1 if n == 0 else x
    p1, p2 = eval_Lm(n - 1, x), eval_Lm(n - 2, x)
    return ((2 * n - 1) * x * p1 - (n - 1) * p2) / n

def eval_Lm_derivative(x, n):
    '''
    Вычисление производной полинома Лежандра
    '''
    L_first = eval_Lm(x, n-1)
    L_second = eval_Lm(x, n)
    return (n/(1-x**2))*(L_first - x*L_second)

def eval_Lm_roots(n):
    '''
    Вычисление заданного количества корней полинома Лежандра
    '''
    ME = machine_epsilon()*10
    roots_res = []
    roots_init_approx = [
        cos(
            (pi*(4*i-1)) / (4*n+2)
        )
        for i in range(1, n+1)
    ]
    for x in roots_init_approx:
        Lm = 1
        while Lm > ME:
            Lm = eval_Lm(x, n)
            Lm_deriv = eval_Lm_derivative(x, n)
            x = x - Lm/Lm_deriv
        roots_res.append(x)
    return roots_res

print(eval_Lm_roots(6))

[0.932469514203152, 0.6612135278716548, 0.23861918608319693, -0.23861918608319693, -0.6612135278716548, -0.932469514203152]


In [188]:
def eval_At_sum(k):
    '''
    Вычисление суммы произведений Ai*ti
    '''
    return 2/(k+1) if k%2==0 else 0

def __build_At_matrix_with_coef_sum(n, Lm_roots):
    '''
    Строим СЛАУ
    '''
    M = []
    for k in range(n):
        row = []
        for ti in Lm_roots:
            row.append(ti**k)
        row.append(eval_At_sum(k))
        M.append(row)
    return M

def __eval_Ai_coefs(M):
    '''
    Решаем СЛАУ, находим коэффициенты Ai
    '''
    n = len(M)
    X = [0 for i in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            ratio = M[j][i]/M[i][i]
            for k in range(n+1):
                M[j][k] = M[j][k] - ratio*M[i][k]
    X[n-1] = M[n-1][n]/M[n-1][n-1]
    for i in range(n-2, -1, -1):
        X[i] = M[i][n]
        for j in range(i+1, n):
            X[i] = X[i] - M[i][j]*X[j]
        X[i] = X[i]/M[i][i]
    return X

def NP_MATRIX_EVAL(n, Lm_roots):
    A = np.array([[ti ** k for ti in Lm_roots] for k in range(n)])
    B = np.array([eval_At_sum(k) for k in range(n)])
    res = np.linalg.solve(A, B)
    return res

def eval_Gauss_integration(func, a, b, N_nodes, x):
    '''
    Проводим численное интегрирование по формуле Гаусса
    '''
    T = eval_Lm_roots(N_nodes)
    A = NP_MATRIX_EVAL(N_nodes, T)
    #A = eval_Ai_coefs(build_At_matrix_with_coef_sum(N_nodes, T))
    #hx = (b-a)/N_nodes
    sum_Aiti = sum([Ai*func(x, (a+b)/2+((b-a)/2)*ti) for (Ai, ti) in zip(A, T)])
    res = ((b-a)/2)*sum_Aiti
    return res
    
#print(build_At_matrix_with_coef_sum(3, eval_Lm_roots(3)))
#print(eval_Ai_coefs(build_At_matrix_with_coef_sum(3, eval_Lm_roots(3))))
print(eval_Gauss_integration(lambda x, y: y**2, 0, 10, 4, 0))
print(1000/3)
print(eval_Gauss_integration(lambda x, y: y**3+y**2-cos(y-1), 0, 10, 4, 0))
print(-sin(9)-sin(1)+8500/3)

333.33333333333326
333.3333333333333
2831.88501769746
2832.079743863284


In [182]:
def eval_Simpson_multiple_integration(func, a, b, c, d, N_nodes, M_nodes):
    '''
    Вычисление кратного интеграла по формуле Симпсона
    '''
    res = 0
    hx = (b-a)/N_nodes
    for i in range(0, int((N_nodes/2)-1), 1):
        res += (
            eval_Gauss_integration(func, c,d, M_nodes, a+2*i*hx)
            + 4*eval_Gauss_integration(func, c,d, M_nodes, a+(2*i+1)*hx)
            + eval_Gauss_integration(func, c,d, M_nodes, a+(2*i+2)*hx)
        )
    return (hx/3)*res
'''
def eval_Simpson_multiple_integration2(func, a, b, c, d, N_nodes, M_nodes):
    
    Вычисление кратного интеграла по формуле Симпсона
    
    res = 0
    h = (b-a)/N_nodes
    x = a
    while x < b:
        next_x = x+h 
        res += (
            eval_Gauss_integration(func, c,d, M_nodes, x)
            + 4*eval_Gauss_integration(func, c,d, M_nodes, x+h/2)
            + eval_Gauss_integration(func, c,d, M_nodes, next_x)
        )
        x = next_x
    return (h/6)*res
'''

def eval_func(t):
    '''
    Интегрируемая функция
    '''
    def inner_func_LR(phi_x, theta_y):
        return (2*cos(theta_y)) / ((1-(sin(theta_y)**2))*(cos(phi_x)**2))
    def inner_func_main(t, LR_func, phi_x, theta_y):
        return (1-exp((-t)*LR_func(phi_x, theta_y))) * cos(theta_y)*sin(theta_y)
    return lambda phi_x, theta_y: inner_func_main(t, inner_func_LR, phi_x, theta_y)

def integrate(t, M, N):
    ef = eval_func(t)
    return (4/pi)*eval_Simpson_multiple_integration(
        ef,
        0, pi/2,
        0, pi/2,
        M, N
    )

print(integrate(1.5, 4, 4))
print(integrate(1.5, 4, 4))

r = 0.05
for i in range(20):
    print(r, "\t", integrate(r, 4, 4))
    r += 0.5

0.49519204748864565
0.49519204748864565
0.05 	 0.10361098846659908
0.55 	 0.4305052654600802
1.05 	 0.4838886846169174
1.55 	 0.49578078145921783
2.05 	 0.4988262561394034
2.55 	 0.4996610962542434
3.05 	 0.4998981331086283
3.55 	 0.49996692000326026
4.05 	 0.49998721992561007
4.55 	 0.49999330097325717
5.05 	 0.4999951486719304
5.55 	 0.49999571786718466
6.05 	 0.49999589556805096
6.55 	 0.49999595176466904
7.05 	 0.49999596975688343
7.55 	 0.4999959755852522
8.05 	 0.4999959774942827
8.55 	 0.4999959781260939
9.05 	 0.4999959783372346
9.55 	 0.4999959784084328


In [6]:
import numpy as np

A = np.array([[1,1,1],[0.7745966692414834, 0.0, -0.7746958686679296],[0.6000000000000001, 0.0, 0.6001536889311581]])
B = np.array([2, 0, 2/3])
np.linalg.solve(A,B)

array([0.55551998, 0.88903117, 0.55544885])

In [ ]:
def bisect_method(func, a, b):
    '''
    Реализация метода половинного деления
    '''
    x = (a+b)/2
    ME = machine_epsilon() * 10
    while ((b-a)/2) < ME and func(x) != 0:
        if func(a)*func(x) < 0:
            b = x
        else:
            a = x
        x = (a+b)/2
    return x
    
def Lm_roots(start_nodes):
    N_nodes = start_nodes
    segment_store = []
    counter = 0
    while len(segment_store) <= start_nodes:
        N_nodes += 1
        delta = 1/N_nodes
        a = -1
        b = a+delta
        segment_store = []
        while b <= 1:
            if eval_Lm(a, N_nodes)*eval_Lm(b, N_nodes) < 0:
                segment_store.append((a,b))
            a = b
            b += delta
            if a<1 and b>1:
                b = 1
        counter += 1
        print(start_nodes, N_nodes, delta, len(segment_store))
        if counter == 20:
            pass
            #break
    return (segment_store)

res = Lm_roots(30)
print(len(res))
print(res)